# Mentoria DiploDatos FAMAF

## Reducción de Emisiones Contaminantes por el Uso de Biocombustibles en Transporte de Cargas y Pasajeros

### Práctico de Análisis y Visualización

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#!pip install pymysql
import pymysql as sql

## Datos propios
_Datasets :_
- Usuarios: registra datos de los usuarios de las bombas
- Vehiculos: registra datos de los vehículos 
- Productos: registra datos de los tipos de combustibles
- Equipos: registra datos de los equipos IoT
- Tanques: registra datos de los tanques de almacenamiento
- Bombas: registra datos de las bombas de suministro de combustible
- Transacciones (mensuales, desde el 2018): registro de los suministros de combustible por cada bomba a cada vehículo
- Historial (mensuales, desde el 2018): registro de los inventarios mensuales de los tanques de combustibles

In [ ]:
# Listado de todas las tablas de la base de datos
# DB_HOST = 'localhost'
# DB_PORT = 33060
# DB_USER = 'superset'
# DB_PASS = 'superset'
# DB_NAME = 'db'
DB_HOST = 'localdocker'
DB_PORT = 3306
DB_USER = 'root'
DB_PASS = 'root_password'
DB_NAME = 'db'

mysqldb = sql.connect(
  host=DB_HOST,
  port=DB_PORT,
  user=DB_USER,
  passwd=DB_PASS,
  database=DB_NAME
)
mysqlcursordb = mysqldb.cursor()
mysqlcursordb.execute("show tables")
lista_tablas = []
for tabla in mysqlcursordb:
    lista_tablas.append(tabla[0])
print(lista_tablas)

In [ ]:
# Listado de todas las tablas de la base de datos
mysqldb = sql.connect(
  host="localdocker",
  port= 3306,  
  user="root",
  passwd="root_password",#"2801",
  database="db"
)
mysqlcursordb = mysqldb.cursor()
mysqlcursordb.execute("show tables")
lista_tablas = []
for tabla in mysqlcursordb:
    lista_tablas.append(tabla[0])
print(lista_tablas)

In [ ]:
# Recuperar los 10 primeros registros de una tabla de la base de datos
mysqlcursordb = mysqldb.cursor()
query_registros = "SELECT * FROM fs_bombas;"
mysqlcursordb.execute(query_registros)
registros  = mysqlcursordb.fetchall()
for reg in registros[0:10]:
    print(reg)

In [ ]:
# Conexión a la base de datos (para MySQL)
from sqlalchemy import create_engine

# Crear motor de conexión sqlalchemy
path_conexion = 'mysql+pymysql://{}:{}@{}:{}/{}'.format(
    DB_USER, DB_PASS, DB_HOST, DB_PORT, DB_NAME
)
conexion = create_engine(path_conexion)

Para este primer práctico estaremos trabajando solo con algunos datasets. En el segundo práctico se incorporará otros datasets, que requieren de mayor limpieza y manipulacion de los datos.

In [ ]:
# Listado de querys para consultas de las tablas a analizar
query_productos = "SELECT * FROM fs_asignacion_producto;"
query_bombas = "SELECT * FROM fs_bombas;"
query_tanques = "SELECT * FROM fs_tanques;"
query_usuarios = "SELECT * FROM fs_usuarios_fuelsentry;"
query_vehiculos = "SELECT * FROM fs_vehiculos;"

In [ ]:
# Obtención de dataframe 'fs_asignacion_producto', 'fs_bombas', 'fs_equipo', 'fs_tanques', 'fs_usuarios_fuelsentry', 'fs_vehiculos'
df_productos = pd.read_sql_query(query_productos, path_conexion)
df_tanques = pd.read_sql_query(query_tanques, path_conexion)
df_usuarios = pd.read_sql_query(query_usuarios, path_conexion)

# También podemos pasar el tipo de formato de las fechas para su conversión al momento de leer los datos
format_string = "%Y-%m-%d %H:%M:%S"
df_bombas = pd.read_sql_query(query_bombas, path_conexion, parse_dates={'fecha_reinicio': format_string})
df_vehiculos = pd.read_sql_query(query_vehiculos, path_conexion)

In [ ]:
pd.set_option('display.max_columns', 50)

### Dataset de asignación de productos:

Contiene información relacionada con los productos de combustible.

Descripción de las columnas:
- 'id_equipo': ID del dispositivo IoT
- 'producto': código del producto
- 'nombre_producto': nombre del producto
- 'codigo': 
- 'precio_litro': precio del producto (en $/l)
- 'coef_var_vol': coeficiente de variación del volumen del producto combustible (en g/ml/°C)
- 'density': densidad del producto (en g/ml)

Densidad (ASTM D 4052): densidad relativa del combustible medido (en g/ml o kg/m3) a la temperatura estándar de 15 °C.

In [ ]:
print(f'Dimensión del dataset (filas, columnas): {df_productos.shape}\n')
print(f'{df_productos.info()}\n')
df_productos.head()

In [ ]:
# Valores o categorias unicas para cada columna
for i in df_productos.columns:
    print(f'{i}: {df_productos[i].nunique()}')

### Dataset de bombas:

Contiene información de las bombas de suministro de combustible.

--> Un punto de suministro del combustible posee uno o varios equipos (dispositivo IoT) 

Descripción de las columnas:
- 'id_bomba': ID de la bomba
- 'id_equipo': ID del dispositivo IoT
- 'bomba': 
- 'producto': tipo de combustible que suministra la bomba
- 'id_tanque': ID del tanque
- 'totalizador': litros suministrados a la fecha de reinicio
- 'fecha_reinicio': fecha de reinicio de la bomba
- 'pulsos_litro': litros que suministra la bomba (por pulso)
- 'tiempo_interrump': tiempo de interrupción del suministro de combustible de la bomba (en segundos)
- 'habilitacion': 
- 'rampa_de_parada'

In [ ]:
print(f'Dimensión del dataset (filas, columnas): {df_bombas.shape}\n')
print(f'{df_bombas.info()}\n')
df_bombas.head()

In [ ]:
# Valores o categorias unicas para cada columna
for i in df_bombas.columns:
    print(f'{i}: {df_bombas[i].nunique()}')

### Dataset de tanques:

Contiene información de los tanques de combustible.

--> Una empresa puede tener más de un tanque

--> Varios tanques pueden estar conectado a un mismo equipo

--> Un tanque posee una o varias bombas

Descripción de las columnas:
- 'id_tanque': ID del tanque
- 'id_equipo': ID del dispositivo IoT 
- 'tanque': tipo de tanque
- 'producto': producto almacenado en el tanque 
- 'capacidad': capacidad del tanque en litros
- 'log_interval': intervalo de tiempo en el cual se registra la medición del volumen del contenido del tanque --> historial del volumen del tanque
- 'nivel_alarma': nivel de contenido del tanque para disparar alarma

In [ ]:
print(f'Dimensión del dataset: {df_tanques.shape}\n')
print(f'{df_tanques.info()}\n')
df_tanques. head()

In [ ]:
for i in df_tanques.columns:
    print(f'{i}: {df_tanques[i].nunique()}')

### Dataset de usuarios:

Contiene información de los usuarios de las bombas de combustible.

Descripción de las columnas:
- 'id_usuario_fuelsentry': ID de registro del usuario
- 'id_equipo':  ID del dispositivo IoT
- 'usuario_fuelsentry': código del usuario de la bomba
- 'departamento': 
- 'codigo': 
- 'totalizador': litros totales suministrados por el usuario de la bomba
- 'cargas_totales': número total de cargas sumnistradas por el usuario

In [ ]:
print(f'Dimensión del dataset: {df_usuarios.shape}\n')
print(f'{df_usuarios.info()}\n')
df_usuarios. head()

In [ ]:
for i in df_usuarios.columns:
    print(f'{i}: {df_usuarios[i].nunique()}')

## Dataset de Vehiculos

Contiene informacion sobre los vehiculos 

Descripcion de las columnas:
- **id_vehiculo:** identificador unico
- **id_equipo:** el vehiculo esta asociado a un equipo, poede decirse que esta asociado a una estacion de carga
- **vehiculo:** corresponde a una identificacion local del vehiculo por parte del equipo.
- **departamento:** agrupacion por departamento. campo propio del cliente.
- **limite:**
- **odometro_inicio:**
- **odometro_fin:**
- **cargas_maximas_dia:**
- **autorizacion:** [campo en desuso]
- **cargas_hoy:** [campo en desuso]
- **cargas_hasta_hoy:** [campo en desuso]
- **ultima_fecha:** [campo en desuso]


In [ ]:
print(f'Dimensión del dataset: {df_vehiculos.shape}\n')
print(f'{df_vehiculos.info()}\n')
df_vehiculos.head()

## Datos públicos:
Bioetanol: producción y ventas (total país)
- Producción y ventas de bioetanol por mes en base a maíz y caña de azúcar (en metros cúbicos).
- Frecuencia de actualización: Mensualmente
- Último cambio: 15 de mayo de 2018

In [ ]:
url_bioetanol_nacional = 'http://datos.minem.gob.ar/dataset/5ce77ad1-c729-42cd-a8b5-2407de005e5b/resource/fd142c49-fa73-4e21-be1f-a10d6d67d05d/download/bioetanol-serie-produccion-y-ventas-total-pais.csv'
bioetanol_nacional=pd.read_csv(url_bioetanol_nacional, parse_dates=['mes'])
print(f'Dimensión del dataset (filas, columnas): {bioetanol_nacional.shape}\n')
print(f'{bioetanol_nacional.info()}\n')
bioetanol_nacional.head()

In [ ]:
bioetanol_nacional['mes'] = pd.to_datetime(bioetanol_nacional['mes'])

In [ ]:
bioetanol_nacional.head()

Biodiesel: producción, ventas y exportaciones, total país
- Producción, ventas y exportaciones de biodiesel por mes en toneladas.
- Frecuencia de actualización: Mensualmente
- Último cambio: 15 de mayo de 2018

In [ ]:
url_biodiesel_nacional = 'http://datos.minem.gob.ar/dataset/5ce77ad1-c729-42cd-a8b5-2407de005e5b/resource/4e04bc74-8625-412c-acc2-48412f2509b4/download/biodiesel-serie-produccion-ventas-y-expo.csv'
# Leer CSV
biodiesel_nacional =  pd.read_csv(url_biodiesel_nacional,parse_dates=['mes'])
biodiesel_nacional

## Análisis a desarrollar:

Indicaciones previas: 
- Para cada planteamiento (excepto el 1), realizar un análisis de los resultados obtenidos y justificar por qué empleó determinado cálculo.

- Para los planteamientos donde utilices gráficos, responder: ¿Qué tipo de gráfico es el recomendado? ¿Por qué? Construya el gráfico y elabore una conclusión del mismo.

Nota: Cuidar los aspectos relevantes de un gráfico (título, nombre de las variables para cada eje, escala de valores en los ejes, colores, fuente de los datos, leyenda, valores a resaltar, etc)

 ### 1- Seleccionar y crear una lista de 5 variables númericas. Comenta si son de tipo discreto o continuo, y por qué.

In [ ]:
# 1.- coheficiente de variacion de volumen.
# `fs_asignacion_producto.coef_var_vol`
# es una variable continua, si bien toma pocos valores 
# esta variable debe considerarse continua ya que corresponde
# a una propiedad fisica
df_productos.coef_var_vol.unique()

In [ ]:
# 2.- totalizador de bomba
# `fs_bombas,totalizador`
# esta variable es continua. corresponde al volumen de 
# combustible despachado por un surtidor. El combustible no se
# despacha en cantidades discretas de volumen por eso debe 
# considerarse continua
pd.to_numeric(df_bombas.totalizador).describe()

In [ ]:
# 3.- intervalo entre logs de tanque.
# `df_tanques.log_interval`
# Variable de tiempo, osea que es continua
pd.to_numeric(df_tanques.log_interval,errors='coerce').dropna().astype(int).unique()

In [ ]:
# 4.- cantidad de cargas totales de un usuario.
# `df_usuarios.cargas_totles`
# Es una variable discreta, un valor entero.
df_usuarios.cargas_totales.astype(int).sample(10)

In [ ]:
# 5.- produccion total de Biodiesel.
# df_biodisel_nac.produccion_total
# Variable continua. Representa el la cantidad de combustible 
# producida en el periodo
biodiesel_nacional.produccion_total

### 2- Determinar algunas medidas estadísticas para las variables seleccionadas.

#### 2.1.- Coheficiente de variacion de volumen

In [ ]:
# coeficiente de variacion de volumen
print("Valores: {}".format(df_productos.coef_var_vol.unique()))
df_productos.groupby('coef_var_vol').count()

Vemos que la variable toma pocos valores. Por conocimiento del campo es conocido que el valor de esta variable lo entrega el productor, por lo tanto podemos decir que cada valor corresponde a un producto diferente.
Por lo tanto el analisis mas util para esta variable sera:
 - Ver a que producto corresponde cada coeficiente ( servira luego para inferir el producto a partir de otros valores). Nota: en el item 5 se muestra porque esto no sera posible
 - Que cantidad de tanques con el mismo producto hay?

Por otro lado podemos descartar el valor 10 porque no hay liquido conocido con ese coeficiente de dilatacion, y el valor 0 tambien puede descargarse como "no valido" ya que la dilatacion es una propiedad fisica

In [ ]:
data = pd.merge(df_productos, df_tanques[['id_equipo','producto','id_tanque']], how='left', left_on=['id_equipo','producto'], right_on=['id_equipo','producto'])
data = data[(data.coef_var_vol > 0) & (data.coef_var_vol < 10)]
data.nombre_producto = data.nombre_producto.str.upper().apply(lambda n: 'DIESEL' if re.match(r'.*(DIESEL|GAS\s{0,1}OIL|EURO|D500).*',n) else n)
data.nombre_producto = data.nombre_producto.apply(lambda n: 'NAFTA' if re.match(r'.*(NAFTA|SUPER|ENERGY).*',n) else n)
data.nombre_producto = data.nombre_producto.apply(lambda n: 'DESCONOCIDO' if re.match(r'.*(PRODUCT|N/A).*',n) else n)

tank_por_prod = data[['nombre_producto','coef_var_vol','id_tanque']].groupby(['nombre_producto','coef_var_vol']).count().sort_values('id_tanque',ascending=False,)
tank_por_prod['porcentaje'] = tank_por_prod.id_tanque.apply(lambda x: (100*x/tank_por_prod.id_tanque.sum()).round(2)) 
print("CANTIDAD DE TANQUES POR PRODUCTO \n[solo se muestran los valores mas relevantes]")
tank_por_prod.head(10)

De la tabla anterior vemos que no conocemos que producto contiene el 65% de los tanques, que solo ~28% de los tanques contienen Diesel.

#### 2.2.- Totalizador de bomba

In [ ]:
df_bombas.totalizador = df_bombas.totalizador.astype(float)
df_bombas.totalizador.describe()

In [ ]:
sns.set_context(context='talk', font_scale=0.8, rc={"lines.linewidth": 3})
fig = plt.figure(figsize=(15,6))
data = df_bombas['totalizador']/1e6
ax = sns.distplot(data, kde=False, norm_hist=False)
ax.set_yscale('log')

plt.title("Distribución de la variable totalizador de bomba")
plt.ylabel('Cantidad de bombas')
plt.xlabel('m3 despachados por bomba')


ax=plt.annotate('Totalizadores en cero en {} bombas\n(*1) Ver aclaración abajo'.format(
                    len(df_bombas[df_bombas['totalizador']==0])
                    )
                 ,xy=(0.65, 0.80), xycoords='axes fraction', fontsize=12)

sns.despine()


**_(*1)_**  Existen 3 posibles motivos por lo cual tenemos tantas bombas con valores 0 o de pocos litros:
* La variable (bomba) no se está utilizando, hay un campo **habilitacion** que está en cero en algunos casos
* Está variable no tiene en cuenta el tiempo por ende podrían ser bombas son nuevas, recién instaladas
* La variable no se está actualizando correctamente
Se concluye que, por el momento, no tiene mucho sentido obtener mediciones estadisticas de esta variables. 
Otra observación importante es que este dato está contenido dentro de las tablas de transacciones (__sis_transa__), donde ahí sí contammos con la variable tiempo y se podrían obtener un análisis mas interesante.

#### 2.3.- Intervalo entre logs de tanque

In [ ]:
data = df_tanques
data.log_interval = pd.to_numeric(data.log_interval,errors='coerce')
data = data.dropna()
data.log_interval = data.log_interval.astype(int)

data.log_interval.describe()

Vemos que al menos el 75% de los valores es menor  o igual que 30.y que el primer y tercer cuartil calen 30 pero esta metrica no es muy ultil, asique hayq que expresarla mejor

In [ ]:
# data[data['log_interval']==0]
interval = df_tanques['log_interval'].round(0).dropna().astype(int)
interval.dtype

In [ ]:
print("sondas con muestreo mayor a 30 minutos: %.1f %%" % (
        (100*len(data[data.log_interval > 30])/len(data)),
    )
)
print("sondas con muestreo menor a 30 minutos: %.1f %%" % (
        (100*len(data[data.log_interval < 30])/len(data)),
    )
)

print("sondas con muestreo cada 10 minutos: %.1f %%" % (
        (100*len(data[data.log_interval == 10])/len(data)),
    )
)

print("sondas con muestreo cada 30 minutos: %.1f %%" % (
        (100*len(data[data.log_interval == 30])/len(data)),
    )
)

print("sondas con muestreo cada 60 minutos: %.1f %%"  % (
        (100*len(data[data.log_interval == 60])/len(data)),
    )
)

fig = plt.figure(figsize=(19,8))
sp1 = plt.subplot('221')
sp1.set_title("Periodos de muestreo menores o iguales a 30 minutos")
ax=sns.distplot(data.log_interval[data.log_interval <= 30], norm_hist=False, kde=False, ax=sp1, color='green')
ax.set_yscale('log')

# ax=sns.distplot(np.log(data['log_interval']), norm_hist=False, kde=False, color='green')

plt.ylabel('Cantidad')
plt.xlabel('Intervalo máximo de mediciones [minutos]')

sp2 = plt.subplot('222')
sp2.set_title('Periodos de muestreo mayores a 30 minutos')
ax = sns.distplot(data.log_interval[data.log_interval > 30], norm_hist=False, kde=False, ax=sp2, color='green')
plt.xlabel('Intervalo máximo de mediciones [minutos]')

ax.set_yscale('log')
# ax.set_xscale('log')


fig.tight_layout()
sns.despine()

Se obserba de la descripcion y grafico anteriores que el ~80% de los equipos toma muestras de nivel de tanque cada 30 minutos, ~9% lo hace cada 60 minutos, y ~5% lo hace cada 10 minutos. El resto de los casos son clientes con requerimientos especificos, dispositivos experimentales o dispositivos mal configurados. Esta consideracion sera importante a la hora de matchear transacciones con historiales en la curacion

#### 2.4.-Cantidad de cargas totales de los usuarios

In [ ]:
fig = plt.figure(figsize=(15,5))


data = df_usuarios
data.cargas_totales = data.cargas_totales.astype(int)
plt.title("Distribución del número de cargas de los usuarios")
print("Rango de la variable: {} - {}".format(data.cargas_totales.min(), data.cargas_totales.max()))
ax=sns.distplot(data.cargas_totales[data.cargas_totales>0],kde=False, norm_hist=False, color='blue')
plt.ylabel('Cantidad de usuarios')
plt.xlabel('Número de cargas por usuario')
ax.set_yscale('log')


Como vemos en el grafico, hay algunos usuarios con muchas cargas mientras otros con pocas, esto no es real, ya que como se muestra en el rango y luego se aprecia en el grafico, esta variable toma valores de hasta tres digitos. Podemos afirmar entonces que la variable es incierta ya que cuando el valor supera 999 vuelve a 000 (overflow), entonces no podemos conocer el valor real. Por otro lado podremos obtener el valor de esta variable en un periodo de tiempo a partir de las tablas de transacciones ya que cada transaccion esta asociada a un usuario. A continuacion se muestra lo mencionado para todo el año 2019

In [ ]:
# Cargo los datos de transaccion de 2019
query = " UNION ".join(
    ["SELECT id_transaccion, id_usuario, fecha FROM sis_transa_2019_{}".format(mes) for mes in range(1,13)]
)
df_transa_2019 = pd.read_sql(sql=query, con=path_conexion)
# elimiro registros espurios (con conocimiento de campo)
df_transa_2019 = df_transa_2019[(df_transa_2019.id_usuario!=0) & (df_transa_2019.fecha!='0000-00-00')]
df_transa_2019.fecha = pd.to_datetime(df_transa_2019.fecha)

In [ ]:
%%time
# Obtengo la cantidad de cargas totales
data = df_transa_2019.groupby(['id_usuario']).count().rename(columns={'fecha':'cantidad'}).sort_index()
# obtengo el periodo activo del usuario
data['fecha_min'] = df_transa_2019.groupby(['id_usuario']).min().sort_index().fecha
data['fecha_max'] = df_transa_2019.groupby(['id_usuario']).max().sort_index().fecha
data['dias'] = (data.fecha_max-data.fecha_min).apply(lambda p: pd.Timedelta(p).days)
del data['id_transaccion']
# elimino los que tuvieron una carga eventual 
data = data[data.dias > 0]
data['cargas_por_dia'] = data.cantidad / data.dias
data

In [ ]:
fig = plt.figure(figsize=(15,5))
# plt.title("Cantidad de cargas en promedio por dia por cada usuario")
plt.title("Distribución del número de cargas promedio por día de los usuarios")
ax=sns.distplot(data.cargas_por_dia, kde=False, color='blue')#, norm_hist=True)
plt.ylabel('Cantidad de usuarios')
plt.xlabel('Número de cargas por día')
ax.set_yscale('log')

plt.show()
data.cargas_por_dia.describe()


La hipotesis que podemos generar partiendo del grafico anterior es que hay dos grupos de usuarios (dos usos para la variable), quienes realizan muchas transacciones de combustible (ej. playeros de estaciones de servicio), es decir que el usuario es quien despacha el combustible y por otro lado tenemos a usuarios que realizan pocas transacciones (ej. choferes), es decir que la variable esta asociada con quien recibe el combustible. 

Apoyandonos en la hipotesis anterior diremos que hay mas usuarios quienes reciben combustible que quienes despachan, el 75 porciento de los usuarios realizan menos de 18 cargas por mes, si descontamos los fines de semana y los feriados esto nos da aproximadamente 1 carga al dia. Intuitivamente podriamos afirmar que un chofer carga combustible dos veces al dia como maximo lo que puede servir como umbral para la diferenciacion de usuarios.

#### 2.5.-Capacidad de los tanques

In [ ]:
#filtro tanques borrados en consola, contiene 'X'
df_tanques = df_tanques[df_tanques['capacidad'].str.contains('x',case=False)==False]

# Capacidad del tanques, variable numerica discreta
df_tanques['capacidad_int64'] = df_tanques['capacidad'].astype('float').round(0).astype('int64')
print(f'Descripción de las capacidades')
print(df_tanques['capacidad_int64'].describe(),'\n')


- La capacidad de los tanques están en litros
- Existen errores en la capacidad asignada, un tanque de almacenamiento de combustible no debería tener menos de 40 litros, si lo comparamos con la capacidad de un automóvil cualquiera estamos en 40 litros, es lógico pensar que almacenar el mismo combustible para distribución los tanques están por encima de estás capacidades


#### 2.6.-Producción de Biodiesel

In [ ]:
fig = plt.figure(figsize=(15,5))

data = biodiesel_nacional

plt.title("Distribucion de la produccion mensual de Biodiesel")
sns.distplot(data.produccion_total, kde=False, color='green')
plt.xlabel('Masa Promedio Mensual [Toneladas]')
plt.show()

data.produccion_total.describe()

La produccion media mensual de biodiesel es de 171.5 KTon con desviacion estandar 70.8KTon , vemos que es asimetrica hacia la derecha, esto se debe a que la produccion ha aumentado a travez de los años y se muestra a continuacion

### 3- Obtenga el número registros en el dataset para cada producto ('producto' de df_bombas). Comente sobre los valores obtenidos.

In [ ]:
print("cantidad de bombas: {}".format(
        len(df_bombas)
    )
)
print("cantidad de productos: {}".format(
        len(df_bombas[['id_equipo','producto']].drop_duplicates())
    )
)

Existen menos productos que bombas, esto nos indica que hay varias bombas que despachan un mismo producto

### 4- Obtener un gráfico para visualizar el comportamiento de los valores de capacidad de los tanques de combustibles. (df_tanques)

In [ ]:
fig = plt.figure(figsize=(15,10))

#filtro capacidades menores a 40 litros
df_tanques_f40 = df_tanques[df_tanques['capacidad_int64']>=40]
df = df_tanques_f40.groupby(['capacidad_int64'], as_index=False).count()


plt.title("Distribución de las capacidades de los tanques")
sns.scatterplot(data=df, x='capacidad_int64', y='id_tanque')
sns.distplot(df_tanques_f40['capacidad_int64'], kde=False, color='red', norm_hist = False, bins=11)
plt.xlabel('Capacidad [Litros]')
plt.ylabel('Cantidad de tanques')
# plt.legend()
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
sns.despine()
plt.show()
df_tanques_f40['capacidad_int64'].describe()



Como muestra el grafico anterior, si bien tenemos tanques de hasta 100kLts. la capacidad del 75 % de los tanques es de hasta 50kLts. Sin embargo es muy representativa la cantidad de almacenamientos de baja capacidad (< 10kLts).

### 5- Obtener un gráfico que muestre y compare el comportamiento del coeficiente de variación de los distintos productos (df_productos). ¿Qué haría para mejorar el gráfico?

In [ ]:
# coeficiente de variacion de volumen
df_productos.coef_var_vol.unique()

In [ ]:
fig = plt.figure(figsize=(15,5))

sp = plt.subplot('111')
ax=sns.countplot(data=df_productos, x='coef_var_vol',ax=sp)
plt.ylabel('Número de productos')
plt.xlabel('Coeficiente de dilatación volumétrica [1/ºC]')
plt.title("Distribución del Coeficiente de dilatación volumétrica de los productos")



data = df_productos.coef_var_vol.dropna()

print(
    "Productos con coheficiente de dilatacion 0.001 : {}%".format(
        (100 * len(data[data == 0.001])/len(data))
    )
)
# hay muchos productos con el mismo coheficiente.

Hay muchos productos con el mismo coeficiente, asique vamos a analizar a que productos corresponde.

In [ ]:
fig = plt.figure(figsize=(15,15))
data = df_productos[df_productos.coef_var_vol == 0.001]
data.nombre_producto = data.nombre_producto.str.upper().apply(lambda n: 'DIESEL' if re.match(r'.*(DIESEL|GAS\s{0,1}OIL|EURO|D500).*',n) else n)
data.nombre_producto = data.nombre_producto.apply(lambda n: 'NAFTA' if re.match(r'.*(NAFTA|SUPER|ENERGY).*',n) else n)
data.nombre_producto = data.nombre_producto.apply(lambda n: 'DESCONOCIDO' if re.match(r'.*(PRODUCT|N/A).*',n) else n)

sp = plt.subplot('211')
sp.set_title("Productos con coheficiente de dilatacion = 0.001 (todos)")

data['clrs'] = np.where(data['nombre_producto'] == 'DESCONOCIDO','green','gray')

sns.countplot(
    data=data, x='nombre_producto',ax=sp,
    order=data.nombre_producto.value_counts().index,
    palette = data['clrs']
)
sp.set_xticklabels(sp.get_xticklabels(), rotation=90)

sp1 = plt.subplot('212')
sp1.set_title("Productos con coheficiente de dilatacion = 0.001 (Solo Conocidos)")
# clrs = ['grey' if x == 'DESCONOCIDO' else 'green' for x in data.nombre_producto ]

                        
sns.countplot(
    data=data[data.nombre_producto != 'DESCONOCIDO'], x='nombre_producto',ax=sp1,
    order=data.nombre_producto[data.nombre_producto != 'DESCONOCIDO'].value_counts().index,
    color='green',
)
sp1.set_xticklabels(sp1.get_xticklabels(), rotation=90)
sp1.set_ylabel('Cantidad')
sp.set_ylabel('Cantidad')

sp.set_xlabel(' ')
sp1.set_xlabel('Nombre del producto')

fig.tight_layout()

Del grafico anterior concluimos que no podemos inferir a que producto real (diesel, gasoil, nafta, etc.) corresponden los que no tienen nombre , ej. "PRODUCTO 1",
ya que son muchos los productos con el mismo coeficiente.

### 6- Analizar la distribución de los litros totales y la cantidad de cargas totales que suministraron los usuarios ('totalizador' y 'cargas_totales' del df_usuarios)

La distribución y análisis de cargas totales se hizo en el punto 2.4 anterior.

In [ ]:
data = df_usuarios
data.cargas_totales = data.cargas_totales.astype(int)
data.totalizador = data.totalizador.astype(float)

fig = plt.figure(figsize=(15,5))
# sp2.set_title("Cantidad de litros despachados")
ax=sns.distplot(data.totalizador,kde=False, norm_hist=False, label='volumen suministrado[Lts]')

plt.ylabel('Cantidad de usuarios')
plt.xlabel('Totalizador [Litros]')
plt.title("Distribución de la Cantidad de litros despachados")
ax.set_yscale('log')

fig.tight_layout()
sns.despine()
plt.show()


data.totalizador.describe()

Como ya se explico en el item 2.4, esta variable esta corrupta, debido a la limitacion de tres cifras se genera un overflow de la variable por eso este dato es incierto, el valor correcto puede obtenerse analizando la tabla de transacciones en un periodo de tiempo.
Lo mismo sucede con la variable totalizador, esta limitada a 5 cifras, esto genera oveflow y la variable se reinicia. no es confiable. Sin embargo podemos obtener el valor de la variable en un periodo de tiempo, idem a la variable cargas_totales.

In [ ]:
# Cargo los datos de transaccion de 2019
query = " UNION ".join(
    ["SELECT id_transaccion, id_usuario, fecha, cantidad FROM sis_transa_2019_{}".format(mes) for mes in range(1,13)]
)
df_transa_2019 = pd.read_sql(sql=query, con=path_conexion)
# elimiro registros espurios (con conocimiento de campo)
df_transa_2019 = df_transa_2019[(df_transa_2019.id_usuario!=0) & (df_transa_2019.fecha!='0000-00-00')]
df_transa_2019.fecha = pd.to_datetime(df_transa_2019.fecha)
df_transa_2019.cantidad = pd.to_numeric(df_transa_2019.cantidad, errors='coerce')
df_transa_2019 = df_transa_2019.dropna()
# Elimino transacciones con cantidad = 0
df_transa_2019 = df_transa_2019[df_transa_2019.cantidad > 0]

In [ ]:
data = df_transa_2019.groupby(['id_usuario']).sum().sort_index()
# obtengo el periodo activo del usuario
data['fecha_min'] = df_transa_2019.groupby(['id_usuario']).min().sort_index().fecha
data['fecha_max'] = df_transa_2019.groupby(['id_usuario']).max().sort_index().fecha
# cantidad de dias que el usuario estuvo activo 
data['dias'] = df_transa_2019[['id_usuario','fecha']].drop_duplicates().groupby(['id_usuario']).count().sort_index().fecha
del data['id_transaccion']
# elimino los que tuvieron una carga eventual 
data = data[data.dias > 0]

data['cantidad_por_dia'] = data.cantidad / data.dias
data['cargas_totales'] = df_transa_2019.groupby(['id_usuario']).count().id_transaccion.sort_index()
data['cargas_por_dia'] = data.cargas_totales / data.dias

data.cargas_por_dia.describe()

Como se menciono en el punto 2.4, es logico pensar que podemos diferenciar los grupos de usuarios umbralizando en cargas_por_dia = 2 es decir que un quien recibe el combustible carga como maximo dos veces al dia. esta hiposis se prueba obserbando que el terver cuartil tiene valor 2.0 

In [ ]:
fig = plt.figure(figsize=(15,10))
data['grupo'] = np.where(data.cargas_por_dia <= 2, "Grupo 1", "Grupo 2")

sp1 = plt.subplot("211")
# sp1.set_title("Cantidad promedio de cargas  por día vs volumen promedio de la transacción por día ")
sp1.set_title("Volumen promedio de cada usuario por día")
sns.scatterplot(data=data, x='cargas_por_dia', y='cantidad_por_dia', hue='grupo', ax=sp1)

sp1.set_ylabel('Volumnen por día [Litros]')
sp1.set_xlabel('Cantidad de cargas por día')

sp2 = plt.subplot("212")
# sp2.set_title("Cantidad promedio de cargas  por día vs volumen promedio de la transacción por día (Grupo 1)")
sp2.set_title("Volumen promedio de cada usuario por día (solo Grupo 1)")
sns.scatterplot(data=data[data.grupo == 'Grupo 1'], x='cargas_por_dia', y='cantidad_por_dia', ax=sp2)

sp2.set_ylabel('Volumen por día [Litros]')
sp2.set_xlabel('Cantidad de cargas por día')

fig.tight_layout()

En el Scaterplot anterior se muestra la relacion entre las variables 'cantidad de cargas por dia' y 'volumen cargado por dia para cada usuario'. para el Grupo dos existe una correlacion positiva de las variables, mientras que para el grupo uno, no hay una correlacion evidente.  
(José) - Yo diría que no hay una correlación visible entre el consumo de cada usuario y la cantidad de veces que carga por día

## Emplear los datos públicos del Bioetanol y Biodiesel: producción y ventas (total país), en las últimos planteamientos:

### 7- Obtener las gráficas de series temporales de la producción de bioetanol en base a caña de azúcar, a maíz y la total, así también para la producción total de biodiesel.

In [ ]:
plt.figure(figsize=(15,6))
sp1=sns.lineplot(data=bioetanol_nacional,
                 x='mes', y='produccion_total',
                 label='Total',
                 ci = None,
                 linewidth = 4
                )

sns.lineplot(data=bioetanol_nacional,
                 x='mes', y='produccion_base_maiz',
                 label='Maíz',
                 ci = None,
                 color = 'orange'
                )

sns.lineplot(data=bioetanol_nacional,
                 x='mes', y='produccion_base_cana_de_azucar',
                 label='Caña de azucar',
                 ci = None,
                 color = 'green'
                )

plt.legend()
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.title('Producción de Bioetanol en Argentina')
plt.ylabel('Volumen Mensual [m3]')
plt.xlabel('Fecha')
sns.despine()
plt.show()
bioetanol_nacional.describe()

En el grafico se muestra la produccion media por mes de bioetanol, se aprecia una alta estacionalidad de la produccion para la caña de azucar, en el primer semestre la produccion es mucho menor que en el segundo.  
Estás conclusiones serán mejor observada en el mapa de calor de mas abajo.

In [ ]:
plt.figure(figsize=(15,6))
sp1=sns.lineplot(data=biodiesel_nacional,
                 x='mes', y='produccion_total',
                 label='Total',
                 ci = None,
                 linewidth = 4,
#                  color = 'blue'
                )

plt.legend()
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.title('Producción de Biodiesel en Argentina')
plt.ylabel('Masa Mensual  [Toneladas]')
plt.xlabel('Fecha')
sns.despine()
plt.show()
biodiesel_nacional.describe()

En el gráfico vemos la produccion total de biodiesel en cada año vemos que tuvo caidas importantes en los años 2013 y 2015 pero tiene una tendencia ascendente, parece haber una caida en 2018, sin embargo no podemos afirmarlo ya que solo tenemos datos hasta mayo de 2018.


### 8- Comparar la distribución de producción total de biodiesel y bioetanol para cada mes y año. ¿En qué años y meses los consumos son más consistentes?
--> Emplear: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html#pandas.pivot_table

In [ ]:
biodiesel = pd.DataFrame()
biodiesel[['fecha','produccion_total']] = biodiesel_nacional[['mes','produccion_total']].copy()
biodiesel['año']=biodiesel['fecha'].dt.year
biodiesel['mes']= biodiesel['fecha'].dt.month
biodiesel['ventas_totales'] = biodiesel_nacional[['ventas_al_corte','otras_ventas_al_mercado_interno','exportaciones']].sum(axis='columns')
biodiesel['ventas_al_corte'] = biodiesel_nacional['ventas_al_corte']
biodiesel['remanente'] = (biodiesel['produccion_total']-biodiesel['ventas_totales']).rolling(12).mean()
biodiesel['produccion_media_anual'] = biodiesel['produccion_total'].rolling(12).mean()
biodiesel['ventas_media_anual'] = biodiesel['ventas_totales'].rolling(12).mean()
# biodiesel['remanente'].rolling(12).mean()

bioetanol = pd.DataFrame()
bioetanol[['fecha','produccion_total']] = bioetanol_nacional[['mes','produccion_total']].copy()
bioetanol['año']=bioetanol['fecha'].dt.year
bioetanol['mes']= bioetanol['fecha'].dt.month
bioetanol['consumo'] = bioetanol_nacional['ventas_totales']


In [ ]:
biodiesel

In [ ]:
plt.figure(figsize=(15,20))
# sns.set(style="whitegrid")

with sns.axes_style("white"):

    sp1 = plt.subplot("211")
    sns.lineplot(data=biodiesel[['fecha','produccion_total','produccion_media_anual']].set_index('fecha'),
                palette = 'summer',
                linewidth = 2,
                ax=sp1
                )


    sns.lineplot(data=biodiesel[['fecha','ventas_totales']].set_index('fecha'),
                palette = 'spring',
                linewidth = 2,
                ax=sp1     
                )

    sns.lineplot(data=biodiesel[['fecha','ventas_al_corte']].set_index('fecha'),
                color = 'blue',
                linewidth = 2,
                ax=sp1     
                )

# sp3 = plt.subplot("212")
with sns.axes_style("whitegrid"):
    sp2 = plt.subplot("212")
    sns.regplot(data=biodiesel, x='produccion_media_anual', 
                y='ventas_media_anual',
                ax=sp2,
               )

sp1.legend()
# sp1.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
sp1.set_title('Producción y Ventas de Biodiesel en Argentina')
sp1.set_ylabel('Masa Mensual  [Toneladas]')
sp1.set_xlabel('Fecha')

sp2.set_title('Correlación entre Ventas/Producción anualizada de biodiesel en Argentina')
sp2.set_ylabel('Consumo/Ventas mensuales [Tn]')
sp2.set_xlabel('Producción media mensual [Tn]')
plt.set_axes_style ="whitegrid"


sns.despine()
plt.show()
# biodiesel_nacional.describe()

En los gráficos podemos observar que hay excelente correlación entre el biodiesel vendido y producido.  
En los años 2014 y 2016 se observa un gran caída tanto en el consumo para corte como para exportación.
En primer gráfico la venta al corte se utiliza para mezclar con el gasoil o diesel tradicional. Tiene un leve aumento con el correr de los años, el cual se tendrá que analizar en detalle mas adelante.

### 9- Obtener un mapa de calor por mes y año, para los valores medios de  producción total de biodiesel y bioetanol.

In [ ]:
exp=pd.pivot(biodiesel,columns='año',index='mes',values='produccion_total')

# para colocar el nombre del mes, se complica porque no quedaba ordenado
# exp = exp.reset_index()
# exp['mes_name'] = pd.to_datetime(exp['mes'], format='%m').dt.month_name()
# exp.set_index('mes_name',drop=True,inplace=True)
# exp.drop('mes',axis='columns',inplace=True)

sns.set_context(context='talk', font_scale=.8)

plt.figure(figsize=(11,7))
sns.heatmap(exp, annot=False, cmap="YlGnBu",
            yticklabels=pd.to_datetime(exp.reset_index()['mes'], format='%m').dt.month_name(locale='Spanish').values)

plt.ylabel('Mes')
plt.xlabel('Año')
plt.title('Producción de biodiesel en Argentina en Toneladas')
plt.xticks(rotation=30)


Como conclusión podemos decir,
* se observa un aumento de la producción con el correr de los años
* entre los meses entre Abril y Noviembre hay mayor producción, quizás debido a situaciones estacionales de cosechas y demás (cabe aclarar que esto es una suposición sin conocimiento alguno)
* hubo bajas de producción en los años 2013 y 2015

In [ ]:
exp=pd.pivot(bioetanol,columns='año',index='mes',values='produccion_total')

plt.figure(figsize=(11,7))
sns.heatmap(exp, annot=False, cmap="YlGnBu",
            yticklabels=pd.to_datetime(exp.reset_index()['mes'], format='%m').dt.month_name(locale='Spanish').values)

plt.ylabel('Mes')
plt.xlabel('Año')
plt.title('Producción de bioetanol en Argentina en m3')
plt.xticks(rotation=30)

Como conclusión podemos decir, se observa un aumento de la producción con el correr de los años y una clara distribución estacional
